In [1]:
from sklearn import tree
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import confusion_matrix
import zipfile
from statsmodels.tsa.deterministic import DeterministicProcess, CalendarFourier
from statsmodels.graphics.tsaplots import plot_pacf
from sklearn.preprocessing import RobustScaler, StandardScaler, Normalizer, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, BaggingRegressor
from sklearn.model_selection import train_test_split

from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import mean_absolute_error

In [2]:
zf = zipfile.ZipFile('C:/Users/Serueja/Downloads/house-prices-advanced-regression-techniques.zip')
test = pd.read_csv(zf.open('test.csv'))
train = pd.read_csv(zf.open('train.csv'))
X = train.drop('SalePrice', axis=1)
y = train['SalePrice']

In [26]:
Z=test.copy()
X.fillna(value=0, inplace=True)
Z.fillna(value=0, inplace=True)
X=pd.get_dummies(X)
Z=pd.get_dummies(Z)
difference = set(X.columns).difference(set(Z.columns))
difference_2 = set(Z.columns).difference(set(X.columns))
X.drop(difference, axis=1, inplace=True)
Z.drop(difference_2, axis=1, inplace=True)

In [20]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [5]:
# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 80, num = 10)]
# Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [2,4]
# Minimum number of samples required to split a node
min_samples_split = [2, 5]
# Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2]
# Method of selecting samples for training each tree
bootstrap = [True, False]

In [6]:
param_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [7]:
model_RFR = RandomForestRegressor(random_state=1)

In [8]:
rf_Grid = GridSearchCV(estimator = model_RFR, param_grid = param_grid, cv = 3, verbose=2, n_jobs = 4)

In [9]:
rf_Grid.fit(X_train, y_train)

Fitting 3 folds for each of 320 candidates, totalling 960 fits


c:\Users\Serueja\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


GridSearchCV(cv=3, estimator=RandomForestRegressor(random_state=1), n_jobs=4,
             param_grid={'bootstrap': [True, False], 'max_depth': [2, 4],
                         'max_features': ['auto', 'sqrt'],
                         'min_samples_leaf': [1, 2],
                         'min_samples_split': [2, 5],
                         'n_estimators': [10, 17, 25, 33, 41, 48, 56, 64, 72,
                                          80]},
             verbose=2)

In [10]:
best_model_RFR = rf_Grid.best_estimator_
best_model_RFR.fit(X_train, y_train)

c:\Users\Serueja\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\ensemble\_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


RandomForestRegressor(max_depth=4, max_features='auto', min_samples_leaf=2,
                      min_samples_split=5, n_estimators=80, random_state=1)

In [12]:
rf_mae = mean_absolute_error(best_model_RFR.predict(X_test), y_test)
print(rf_mae)

22539.35457621816


In [13]:
rf_score_train = best_model_RFR.score(X_train, y_train)
rf_score_test = best_model_RFR.score(X_test, y_test)
print(rf_score_train)
print(rf_score_test)

0.8674787726436338
0.8285471243382312


In [28]:
rfr_prediction=best_model_RFR.predict(Z)
rfr_prediction_series=pd.Series(rfr_prediction)
sub = pd.DataFrame({'Id':test['Id'], 'SalePrice':rfr_prediction_series})
sub.to_csv('submission_1.csv', index=False)